In [1]:
import pandas as pd

step 1: Reading the CSV files

df1 = pd.read_csv('C:/Users/Naseri/Downloads/CENSUS_MHI_STATE.csv')
df2 = pd.read_csv('C:/Users/Naseri/Downloads/CENSUS_POPULATION_STATE.csv')
df3 = pd.read_csv('C:/Users/Naseri/Downloads/REDFIN_MEDIAN_SALE_PRICE.csv')

step 2: Convert both datasets to long format using melt

In [3]:
df1_cleaned = df1.drop(index=0).copy()

In [6]:
df1_cleaned.rename(columns={df1_cleaned.columns[0]: "Group"}, inplace=True)

In [27]:
df1_long = df1_cleaned.melt(
id_vars=["Group"],
var_name="Full Category",
value_name="Value"
)

In [28]:
split_cols = df1_long["Full Category"].str.split("!!", expand=True)
split_cols.columns = ["State", "Metric Type", "Metric"]

In [29]:
df1_long = df1_long.join(split_cols)
df1_long.drop(columns=["Full Category"], inplace=True)

In [30]:
df1_long.head()

,Group,Value,State,Metric Type,Metric
0,Households,"1,969,105",Alabama,Number,Estimate
1,One race--,NaN,Alabama,Number,Estimate
2,White,"1,327,559",Alabama,Number,Estimate
3,Black or African American,"513,880",Alabama,Number,Estimate
4,American Indian and Alaska Native,"7,795",Alabama,Number,Estimate


Converting df2 to long format

In [15]:
id_vars = ['Label (Grouping)']
value_vars = [col for col in df2.columns
             if col not in id_vars]

In [16]:
df2_long = df2.melt(id_vars=id_vars,value_vars=value_vars,
                   var_name='State_Attribute',
                   value_name='Value')

In [18]:
df2_long[['State', 'Attribute']] = df2_long['State_Attribute'].str.split('!!', expand=True)

In [19]:
df2_long = df2_long[['State', 'Attribute', 'Label (Grouping)', 'Value']]

In [20]:
df2_long.head(10)

,State,Attribute,Label (Grouping),Value
0,Alabama,Estimate,SEX AND AGE,NaN
1,Alabama,Estimate,Total population,"5,108,468"
2,Alabama,Estimate,Male,"2,471,801"
3,Alabama,Estimate,Female,"2,636,667"
4,Alabama,Estimate,Sex ratio (males per 100 females),93.7
5,Alabama,Estimate,Under 5 years,"288,019"
6,Alabama,Estimate,5 to 9 years,"305,731"
7,Alabama,Estimate,10 to 14 years,"331,262"
8,Alabama,Estimate,15 to 19 years,"350,694"
9,Alabama,Estimate,20 to 24 years,"333,795"


converting df3 to long format

In [21]:
df3_cleaned = df3.drop(index=0).copy()

In [22]:
df3_cleaned.rename(columns={df3_cleaned.columns[0]: 'Region'}, inplace=True)

In [23]:
df3_long = df3_cleaned.melt(id_vars=["Region"], var_name="Month", value_name="Median Sale Price")

In [24]:
df3_long['Median Sale Price'] = df3_long['Median Sale Price'].replace({'\$': '', 'K': ''}, regex=True).astype(float)

In [25]:
df3_long.head()

,Region,Month,Median Sale Price
0,Alabama,Month of Period End,131.0
1,Alaska,Month of Period End,240.0
2,Arizona,Month of Period End,125.0
3,Arkansas,Month of Period End,100.0
4,California,Month of Period End,281.0


 Step 3: Merging the two dataframes based on the "State" column
Merge df1 and df2 on the State column (outer join to retain all data)

In [45]:
combined_data = pd.merge(df1_long, df2_long, on="State", how="outer")

In [32]:
combined_data.head()

,Group,Value_x,State,Metric Type,Metric,Attribute,Label (Grouping),Value_y
0,Households,"1,969,105",Alabama,Number,Estimate,Estimate,SEX AND AGE,NaN
1,Households,"1,969,105",Alabama,Number,Estimate,Estimate,Total population,"5,108,468"
2,Households,"1,969,105",Alabama,Number,Estimate,Estimate,Male,"2,471,801"
3,Households,"1,969,105",Alabama,Number,Estimate,Estimate,Female,"2,636,667"
4,Households,"1,969,105",Alabama,Number,Estimate,Estimate,Sex ratio (males per 100 females),93.7


In [75]:
combined_data.to_csv("combined_output.csv", index=False)

print("Combined data saved to combined_output.csv")

Combined data saved to combined_output.csv
